第二问-在给予的第一问的回答中选一个题并且做一个报道：What is the profitability profile across different vehicle types, distances, and locations? Give the table used and also show some visualization.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [26]:
df = pd.read_csv("ncr_ride_bookings.csv")

In [24]:

valid_df = df.dropna(subset=["Booking Value", "Ride Distance"])

profitability_table = valid_df.groupby("Vehicle Type").agg(
    Avg_Booking_Value=("Booking Value", "mean"),
    Avg_Distance_km=("Ride Distance", "mean"),
    Ride_Count=("Booking Value", "count"),
    Total_Revenue=("Booking Value", "sum")
)
profitability_table["Revenue_per_km"] = profitability_table["Total_Revenue"] / (
    profitability_table["Avg_Distance_km"] * profitability_table["Ride_Count"]
)

profitability_table = profitability_table[[
    "Avg_Booking_Value", "Avg_Distance_km", "Revenue_per_km", "Ride_Count", "Total_Revenue"
]]

profitability_table = profitability_table.round({
    "Avg_Booking_Value": 2,
    "Avg_Distance_km": 2,
    "Revenue_per_km": 2,
    "Total_Revenue": 2
})
profitability_table


,Avg_Booking_Value,Avg_Distance_km,Revenue_per_km,Ride_Count,Total_Revenue
Vehicle Type,,,,,
Auto,506.73,24.62,20.59,25415,12878422.0
Bike,510.20,24.65,20.70,15362,7837697.0
Go Mini,507.68,24.61,20.63,20364,10338496.0
Go Sedan,511.50,24.61,20.78,18318,9369719.0
Premier Sedan,509.57,24.60,20.72,12315,6275332.0
Uber XL,501.82,24.40,20.56,3045,1528032.0
eBike,503.90,24.99,20.16,7181,3618485.0


In [27]:
completed_rides = df[df['Booking Status'] == 'Completed'].copy()
completed_rides = completed_rides.dropna(subset=['Booking Value', 'Ride Distance'])
completed_rides['Revenue_per_km'] = completed_rides['Booking Value'] / completed_rides['Ride Distance']
vehicle_summary = completed_rides.groupby('Vehicle Type').agg({
    'Booking Value': ['mean', 'count'],
    'Ride Distance': 'mean',
    'Revenue_per_km': 'mean'
}).round(2)
vehicle_summary.columns = ['Avg_Booking_Value', 'Total_Rides', 'Avg_Distance_km', 'Avg_Revenue_per_km']
vehicle_summary = vehicle_summary.sort_values('Avg_Revenue_per_km', ascending=False)
total_revenue_by_type = completed_rides.groupby('Vehicle Type')['Booking Value'].sum().round(0)
vehicle_summary['Total_Revenue'] = total_revenue_by_type

vehicle_summary


,Avg_Booking_Value,Total_Rides,Avg_Distance_km,Avg_Revenue_per_km,Total_Revenue
Vehicle Type,,,,,
Go Sedan,512.03,16676,25.98,34.66,8538560.0
Auto,506.48,23155,25.99,34.00,11727615.0
Bike,509.11,14034,26.00,33.89,7144913.0
Uber XL,505.30,2783,25.72,33.85,1406256.0
Premier Sedan,509.57,11252,25.95,33.73,5733655.0
Go Mini,507.38,18549,25.99,33.49,9411418.0
eBike,503.46,6551,26.34,32.47,3298157.0


In [28]:
filter_df = df[df["Ride Distance"] >=  24.64]

In [31]:
filter_df[filter_df["Driver Ratings"] >= 4.5]["Customer Rating"].mean()

np.float64(4.406456405754035)

In [33]:
high_rating_df = filter_df[filter_df["Driver Ratings"] >= 4.5]
low_rating_df = filter_df[filter_df["Driver Ratings"] < 4.5]
def calc_metrics(data):
    return {
        "Customer Rating": data["Customer Rating"].mean(),
        "Average VTAT (minutes)": data["Avg VTAT"].mean(),
        "Average CTAT (minutes)": data["Avg CTAT"].mean(),
        "Revenue per km (₹)": data["Booking Value"].sum() / data["Ride Distance"].sum(),
        "Booking Value (₹)": data["Booking Value"].mean()
    }

metrics_df = pd.DataFrame({
    "Driver Rating >= 4.5": calc_metrics(high_rating_df),
    "Driver Rating < 4.5": calc_metrics(low_rating_df)
}).T

metrics_df.T


,Driver Rating >= 4.5,Driver Rating < 4.5
Customer Rating,4.406456,4.407165
Average VTAT (minutes),8.531917,8.513118
Average CTAT (minutes),30.023442,30.042688
Revenue per km (₹),13.680096,13.655592
Booking Value (₹),509.888836,509.034199


In [43]:

filter_df = df[df["Ride Distance"] >=  24.64]

In [44]:
high_rating_df = filter_df[filter_df["Driver Ratings"] >= 4.5]
low_rating_df = filter_df[filter_df["Driver Ratings"] < 4.5]
def calc_metrics(data):
    return {
        "Customer Rating": data["Customer Rating"].mean(),
        "Average VTAT (minutes)": data["Avg VTAT"].mean(),
        "Average CTAT (minutes)": data["Avg CTAT"].mean(),
        "Revenue per km (₹)": data["Booking Value"].sum() / data["Ride Distance"].sum(),
        "Revenue per km (alt ₹)": data["Booking Value"].mean() / data["Ride Distance"].mean(),
        "Booking Value (₹)": data["Booking Value"].mean()
    }

metrics_df = pd.DataFrame({
    "Driver Rating >= 4.5": calc_metrics(high_rating_df),
    "Driver Rating < 4.5": calc_metrics(low_rating_df)
}).T

metrics_df.T

,Driver Rating >= 4.5,Driver Rating < 4.5
Customer Rating,4.406456,4.407165
Average VTAT (minutes),8.531917,8.513118
Average CTAT (minutes),30.023442,30.042688
Revenue per km (₹),13.680096,13.655592
Revenue per km (alt ₹),13.680096,13.655592
Booking Value (₹),509.888836,509.034199


In [41]:
from scipy import stats

# Define long-distance trips (above average distance)
long_distance_threshold = 24.64  # 24.64 km
print(f"Long-distance threshold: {long_distance_threshold:.2f} km")

# Filter for rides with available distance data
df_with_distance = df.dropna(subset=['Ride Distance']).copy()
print(f"Rides with distance data: {len(df_with_distance)}")

# Create long-distance trip indicator
df_with_distance['is_long_distance'] = df_with_distance['Ride Distance'] > long_distance_threshold
long_distance_trips = df_with_distance[df_with_distance['is_long_distance']].copy()
print(f"Long-distance trips: {len(long_distance_trips)}")

# Filter for rides with driver ratings (for driver analysis)
long_distance_with_ratings = long_distance_trips.dropna(subset=['Driver Ratings']).copy()
print(f"Long-distance trips with driver ratings: {len(long_distance_with_ratings)}")

# Define driver rating groups
long_distance_with_ratings['driver_rating_group'] = long_distance_with_ratings['Driver Ratings'].apply(
    lambda x: 'High-rated (4.5+)' if x >= 4.5 else 'Other drivers (<4.5)'
)

# Analyze performance metrics for completed long-distance trips by driver rating group
performance_analysis = []


for group in ['High-rated (4.5+)', 'Other drivers (<4.5)']:
    group_data = long_distance_with_ratings[long_distance_with_ratings['driver_rating_group'] == group]
    
    # Basic metrics
    total_trips = len(group_data)
    avg_customer_rating = group_data['Customer Rating'].mean()
    avg_vtat = group_data['Avg VTAT'].mean()
    avg_ctat = group_data['Avg CTAT'].mean()
    avg_booking_value = group_data['Booking Value'].mean()
    avg_distance = group_data['Ride Distance'].mean()
    revenue_per_km = avg_booking_value / avg_distance
    
    # Payment method distribution
    payment_methods = group_data['Payment Method'].value_counts(normalize=True) * 100
    
    performance_analysis.append({
        'Driver Group': group,
        'Total Trips': total_trips,
        'Avg Driver Rating': group_data['Driver Ratings'].mean(),
        'Avg Customer Rating': avg_customer_rating,
        'Avg VTAT (min)': avg_vtat,
        'Avg CTAT (min)': avg_ctat,
        'Avg Booking Value (₹)': avg_booking_value,
        'Avg Distance (km)': avg_distance,
        'Revenue per km (₹)': revenue_per_km,
        'Cash Payment (%)': payment_methods.get('Cash', 0),
        'Card Payment (%)': payment_methods.get('Card', 0),
        'Digital Wallet (%)': payment_methods.get('Digital Wallet', 0)
    })

# Statistical significance test for revenue per km
high_rated = long_distance_with_ratings[long_distance_with_ratings['driver_rating_group'] == 'High-rated (4.5+)']
other_drivers = long_distance_with_ratings[long_distance_with_ratings['driver_rating_group'] == 'Other drivers (<4.5)']

# Calculate revenue per km for each group
high_rated_rev_per_km = high_rated['Booking Value'] / high_rated['Ride Distance']
other_rev_per_km = other_drivers['Booking Value'] / other_drivers['Ride Distance']

# Run statistical test
t_stat, p_val = stats.ttest_ind(high_rated_rev_per_km, other_rev_per_km)
print(f"Revenue per km: High-rated ({high_rated_rev_per_km.mean():.2f}) vs Other ({other_rev_per_km.mean():.2f})")
print(f"T-statistic: {t_stat:.3f}, P-value: {p_val:.6f}")

Long-distance threshold: 24.64 km
Rides with distance data: 102000
Long-distance trips: 49168
Long-distance trips with driver ratings: 49168
Revenue per km: High-rated (14.20) vs Other (14.22)
T-statistic: -0.215, P-value: 0.829710
